In [ ]:
%pip install langchain langchain-openai langchain_chroma langchain_community langchainhub beautifulsoup4


In [ ]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AIKEY"] = getpass.getpass()

token = os.getenv("AIKEY_SECOND")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, base_url=endpoint, openai_api_key=token)


In [ ]:
import bs4
#from bs4 import BeautifulSoup
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lt.wikipedia.org/wiki/Vilnius/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            #class_=("post-content", "post-title", "post-header")
            id="bodyContent"
        )
    ),
)

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embedding = OpenAIEmbeddings(
    base_url="https://models.inference.ai.azure.com",
    model="text-embedding-3-large",
    openai_api_key=openai_api_key
)

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

#print(prompt)

In [ ]:
def format_docs(docs):
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Convolutional Neural Networks?")